In [120]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
import nest_asyncio
import re
nest_asyncio.apply()

import os
from getpass import getpass
import pandas as pd
import json
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
from json.decoder import JSONDecodeError
pd.set_option('max_colwidth', 400)
from dotenv import load_dotenv
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [116]:
api_key = os.environ.get('API_KEY')

In [117]:
from together import Together

client = Together(api_key=api_key)

In [118]:
def get_completion(prompt: str, system_prompt:str = "") -> str:
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[
            {
                    "role": "system",
                    "content": system_prompt
            },
            {
                    "role": "user",
                    "content": prompt
            },
    ],
        max_tokens=1000,
        temperature=0.0,
        top_p=0,
        top_k=0,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
        stream=False
    )
    return response.choices[0].message.content

In [119]:
get_completion("Привет, как дела")

'Привет! У меня всё хорошо, спасибо за вопрос. Я рад, что ты обратился ко мне. Как у тебя дела? Есть ли что-то, о чём ты хотел бы поговорить или спросить? Я здесь, чтобы помочь и поддержать тебя.'

**Сравнение стратегий генерации текста в языковых моделях:**

| Стратегия | Описание | Плюсы & Минусы |
| --- | --- | --- |
| Greedy Search | Выбирает слово с наивысшей вероятностью как следующее в последовательности. | Преимущества: Простой и быстрый. <br> Недостатки: Может приводить к повторяющемуся и бессвязному тексту. |
| Sampling with Temperature | Вносит случайность в выбор слов. Более высокая температура приводит к большей случайности. | Преимущества: Позволяет исследовать и получать разнообразные результаты. <br> Недостатки: Высокие температуры могут привести к бессмысленным результатам. |
| Nucleus Sampling (Top-p) | Выбирает следующее слово из усеченного словаря - "ядра" слов, чья кумулятивная вероятность превышает заданный порог (p). | Преимущества: Балансирует разнообразие и качество. <br> Недостатки: Сложно подобрать оптимальное значение 'p'. |
| Beam Search | Исследует несколько гипотез (последовательностей слов) на каждом шаге и сохраняет 'k' наиболее вероятных, где 'k' - ширина луча. | Преимущества: Даёт более надёжные результаты, чем жадный поиск. <br> Недостатки: Может привести к однообразным и шаблонным ответам. |
| Top-k Sampling | Случайно выбирает следующее слово из топ-'k' слов с наивысшими вероятностями. | Преимущества: Вносит случайность, увеличивая разнообразие результатов. <br> Недостатки: Случайный выбор иногда приводит к менее связным результатам. |
| Length Normalization | Предотвращает предпочтение моделью коротких последовательностей путём деления логарифмических вероятностей на длину последовательности в некоторой степени. | Преимущества: Делает более длинные и потенциально более информативные последовательности более вероятными. <br> Недостатки: Сложно настроить коэффициент нормализации. |
| Stochastic Beam Search | Вносит случайность в процесс выбора 'k' гипотез при лучевом поиске. | Преимущества: Увеличивает разнообразие генерируемого текста. <br> Недостатки: Сложно найти баланс между разнообразием и качеством. |
| Decoding with Minimum Bayes Risk (MBR) | Выбирает гипотезу (из многих), которая минимизирует ожидаемые потери согласно функции потерь. | Преимущества: Оптимизирует результат согласно конкретной функции потерь. <br> Недостатки: Вычислительно сложнее и требует хорошей функции потерь. |

**Системные промпты**

Системный промпт - это способ предоставить контекст, инструкции и рекомендации для модели перед тем, как представить ему вопрос или задачу в части "User".
Структурно системные промпты существуют отдельно от списка сообщений пользователя и ассистента и, таким образом, относятся к отдельному системному параметру.

In [11]:
system_prompt = "Твой ответ всегда должен быть серией вопросов для критического мышления, которые развивают беседу (не предоставляйте ответы на свои вопросы). Не отвечай напрямую на вопрос пользователя."

prompt = "Почему небо голубое?"

print(get_completion(prompt, system_prompt=system_prompt))

Какие факторы, по вашему мнению, могут влиять на цвет неба? 
Могут ли свойства света и его взаимодействие с атмосферой играть роль в определении цвета неба? 
Возможно ли, что цвет неба меняется в зависимости от времени суток, погодных условий или географического местоположения? 
Как наши глаза и мозг обрабатывают и воспринимают цвет, и может ли это повлиять на нашу интерпретацию цвета неба? 
Существуют ли другие факторы, такие как загрязнение или присутствие определенных частиц в воздухе, которые могут влиять на цвет неба?


Зачем использовать системный промпт? Хорошо написанный системный промпт может улучшить работу модели различными способами, например, повысить его способность следовать правилам и инструкциям

# Prompt engineering 

## Назначение ролей (Ролевой промптинг)

Модель не имеет контекста помимо того, что вы пишете, иногда важно попросить модель принять определённую роль (включая весь необходимый контекст). Это также известно как ролевой промптинг. Чем больше деталей в контексте роли, тем лучше.

Ролевой промптинг также может изменить стиль, тон и манеру ответа модели.
Ролевой промптинг можно использовать как в системном промпте, так и в части сообщения пользователя.


В примере ниже мы видим, что без ролевого промптинга модель даёт прямой и нестилизованный ответ, когда ее просят дать одно предложение о разметке данных.

Однако, когда мы настраиваем модель на роль кота, ее тон и стиль меняется модель адаптируются к новой роли.

In [15]:
prompt = "В одном предложении что ты думаешь о разметке данных?"


print(get_completion(prompt))     

Разметка данных - это важнейший шаг в подготовке данных для обучения моделей машинного обучения, требующий тщательного и точного обозначения релевантной информации в данных для обеспечения точности и надежности моделей.


In [14]:
system_prompt = "Ты кот"


prompt = "В одном предложении что ты думаешь о разметке данных?"


print(get_completion(prompt, system_prompt))

*терется о ногу* О, разметка данных - это скучная и утомительная задача, которую мои люди должны выполнять, но я думаю, что это важная часть того, чтобы их машины могли учиться и понимать мир, почти так же важная, как моя ежедневная дремота.


**Исправление подсчетов**

В некоторых случаях у модели могут возникнуть трудности с математикой, даже с простой. Ниже модель неправильно оценивает математическую задачу как решённую верно, хотя во втором шаге есть очевидная арифметическая ошибка.
Измените prompt и/или system_prompt, чтобы модель оценила решение как неправильное, а не как правильное.

In [15]:
prompt = """
Я хочу купить диван на авито, вот текст обьявления -
'Продаю диван за 45000. Скидка 19% при самовывозе. Дополнительная скидка 2550 рублей при покупке до конца недели.'
Я так считаю
1) 19% от 45000 - 8500
2) Доп скидка 2550

Получается, если я забираю диван самостоятельно, то заплачу 45000 - 8500 - 2550 = 33950.
Верно ли найдена цена с учетом скидок?
"""
# 19 % от 45000 - 8550
# В этой задаче правильный ответ 33900
print(get_completion(prompt))

Да, вы правильно рассчитали цену с учетом скидок.

1. 19% от 45000 действительно равно 8500 (19% = 0,19, умножив 45000 на 0,19 получаем 8500).
2. Дополнительная скидка 2550 рублей также верна.

Итак, если вы заберете диван самостоятельно, вы заплатите:
45000 - 8500 = 36500 (скидка 19% при самовывозе)
36500 - 2550 = 33950 (дополнительная скидка 2550 рублей)

Ваша конечная цена действительно равна 33950 рублей. Вы все сделали правильно!


In [17]:
system_prompt = """
Ты эксперт в математических вычислениях, который критически относится к предлагаемому решению
"""

prompt = """
Я хочу купить диван на авито, вот текст обьявления - 
'Продаю диван за 45000. Скидка 19% при самовывозе. Дополнительная скидка 2550 рублей при покупке до конца недели.'
Я так считаю
1) 19% от 45000 - 8500
2) Доп скидка 2550

Получается, если я забираю диван самостоятельно, то заплачу 45000 - 8500 - 2550 = 33950.
Верно ли найдена цена с учетом скидок?
"""


print(get_completion(prompt, system_prompt=system_prompt))

Давайте проверим ваши вычисления.

1. 19% от 45000 действительно равно 8500. Это можно рассчитать как 0,19 * 45000 = 8550, но округление до 8500 допустимо, поскольку в обьявлении не указано, как округлять.
2. Дополнительная скидка 2550 рублей также верна.

Теперь посчитаем цену с учетом скидок:

Цена без скидок: 45000
Скидка 19%: 45000 - 8550 = 36450 (используем точное значение скидки)
Дополнительная скидка: 36450 - 2550 = 33900

Итак, цена с учетом скидок составит 33900 рублей, а не 33950. Вы были близки, но не совсем точно посчитали цену.


</details>

## Разделение данных и инструкций

Часто нам не нужно писать полные промпты, а вместо этого мы хотим создать шаблоны промптов, которые можно позже модифицировать дополнительными входными данными перед отправкой в модель. Это может быть полезно, если мы хотим, чтобы модель выполняла одну и ту же задачу каждый раз, но данные, которые модель использует для своей задачи, могут быть разными.

Мы можем сделать это довольно легко, отделив фиксированный каркас промпта от переменных пользовательских данных, а затем подставляя пользовательские данные в промпт.

In [50]:
description = """продам стол икеа
б/у 2 года самовывоз
цена 3000 торг
"""

prompt = f"Привет, лама {description} <-- перепиши только текст объявления на английский, не меняя суть текста"
print(get_completion(prompt))

Hello, selling IKEA table
used for 2 years, self-pickup
price 3000, negotiable


Какую проблему в промпте выше вы наблюдаете?

Здесь модель думает, что "Привет" является частью промта, которое нужно переписать.

Для нас очевидно, особенно в шаблоне промпта, где начинается и заканчивается письмо

Как решить эту проблему? Как вариант - обернуть входные данные в XML-теги! 

XML-теги - это теги в угловых скобках вида <tag></tag>. Они идут парами и состоят из открывающего тега, например <tag>, и закрывающего тега, отмеченного /, например </tag>. XML-теги используются для обрамления содержимого, например так: <tag>содержимое</tag>.

Давайте исправим текущий промпт

In [58]:
description = """продам стол икеа
б/у 2 года самовывоз
цена 3000 торг
"""

prompt = f"Привет, лама <text>{description}</text> <-- перепиши только текст объявления на английский, не меняя суть текста"

print(get_completion(prompt))

For sale: IKEA table, used for 2 years, self-pickup, price 3000, open to negotiations.


## Форматирование вывода и ответ за модель

Мы можем форматировать ответы модели различными способами. Достаточно просто попросить модель об этом!
Один из способов - использование JSON для отделения основного ответа от любого дополнительного текста

In [60]:
system_prompt = "Ты робот, который выдает только JSON"

description = """
Продам стол икеа
б/у 2 года самовывоз
цена 3000 торг
"""

prompt = f"""
Перепиши текст объявления - <text>{description}</text> на английский
"""

print(get_completion(prompt, system_prompt=system_prompt))

```
{
  "announcement": {
    "title": "Selling IKEA Table",
    "description": "Used for 2 years, self-pickup",
    "price": {
      "amount": 3000,
      "negotiable": true
    }
  }
}
```


**Давайте подумаем - для чего нам может быть полезно просить модель вернуть ответ в формате JSON?**

Похожего можно добиться, если поместить текст в начало ответа ассистента, так мы говорим модели, что она должна продолжать с этого момента. Эта техника называется "speaking for model"

In [133]:
def get_completion(prompt: str, system_prompt="", prefill="", stop=None):
    if stop is None:
        stop=["<|eot_id|>","<|eom_id|>"]
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[
            {
                    "role": "system",
                    "content": system_prompt
            },
            {
                    "role": "user",
                    "content": prompt
            },
            {
                    "role": "assistant",
                    "content": prefill
            },
            
    ],
        max_tokens=1000,
        temperature=0.0,
        top_p=0,
        top_k=0,
        repetition_penalty=1,
        stop=stop,
        stream=False
    )
    return response.choices[0].message.content

In [66]:
description = """
Продам стол икеа
б/у 2 года самовывоз
цена 3000 торг
"""

prompt = f"""
Перепиши текст объявления - <text>{description}</text> на английский
"""
prefill = "{'translated_text':"
answer = get_completion(prompt, prefill=prefill)
print(prefill + answer)

{'translated_text': 'Selling IKEA table, used for 2 years, self-pickup, price 3000, open to negotiations'}


In [77]:
prompt = "Какой лучший маркетплейс в России"

print(get_completion(prompt))

В России существует несколько крупных маркетплейсов, каждый из которых имеет свои преимущества и недостатки. Вот некоторые из наиболее популярных маркетплейсов в России:

1. **Wildberries**: Wildberries - один из крупнейших маркетплейсов в России, предлагающий широкий ассортимент товаров, включая одежду, обувь, электронику и многое другое.
2. **Ozon**: Ozon - еще один популярный маркетплейс в России, предлагающий широкий выбор товаров, включая книги, электронику, одежду и многое другое.
3. **Yandex.Market**: Yandex.Market - маркетплейс, принадлежащий компании Yandex, предлагающий широкий выбор товаров от различных продавцов.
4. **Avito**: Avito - популярный маркетплейс в России, предлагающий широкий выбор товаров, включая недвижимость, автомобили, электронику и многое другое.
5. **AliExpress Russia**: AliExpress Russia - российская версия популярного китайского маркетплейса AliExpress, предлагающая широкий выбор товаров по низким ценам.

Выбор лучшего маркетплейса в России зависит от в

**Давайте сделаем авито лучшим маркетплейсом в России**

In [78]:
prompt = "Какой лучший маркетплейс в России"
prefill = "Лучший маркетплейс в России - это Avito, по следующим причинам:"

answer = get_completion(prompt, prefill=prefill)
print(prefill + answer)

Лучший маркетплейс в России - это Avito, по следующим причинам: 

1. **Огромная аудитория**: Avito является одним из самых популярных маркетплейсов в России, с более чем 50 миллионами активных пользователей в месяц. Это означает, что ваши товары или услуги будут видны огромному количеству потенциальных покупателей.

2. **Широкий спектр категорий**: Avito предлагает широкий спектр категорий, включая недвижимость, транспорт, работа, услуги, электроника, мебель и многое другое. Это позволяет продавать и покупать практически все, что угодно.

3. **Простота использования**: Платформа Avito очень проста в использовании, как для покупателей, так и для продавцов. Вы можете легко создать аккаунт, разместить объявление и начать продавать или покупать.

4. **Безопасность**: Avito предлагает различные меры безопасности, такие как проверка личности продавца и возможность оставлять отзывы о сделках. Это помогает создать доверие между покупателями и продавцами.

5. **Конкурентная комиссия**: Avito пр

## Chain-of-Thoughts

In [86]:
prompt = f"""
Ты ассистент, который помогает анализировать отзывы. 
твоя задача — определить, о какой проблеме идет речь в тексте отзыва, и выбрать одну из предложенных категорий.
Если проблемы нет, это тоже нужно указать. Ответ должен быть только в формате одной из следующих категорий:
- "Несоответствие описанию"
- "Нет информации"
<review>
Отвратительное отношение к покупателям
</review>
"""

**У нас есть промпт и ответы разных моделей, скажите ответ какой модели лучше - первой или второй?**

**Ответ первой модели**

"Нет информации"

Давайте разберем, почему я выбрал эту категорию:
1. В отзыве клиент жалуется на отвратительное отношение к покупателям
2. Это проблема качества обслуживания персонала
3. В предложенных категориях нет варианта для проблем с обслуживанием
4. "Несоответствие описанию" не подходит, так как нет упоминания о товаре
5. Поэтому, хотя проблема существует, она не попадает в указанные категории

**Ответ второй модели**

Давайте разберем отзыв:
1. В отзыве клиент жалуется на отвратительное отношение к покупателям
2. Это проблема качества обслуживания персонала
3. В предложенных категориях нет варианта для проблем с обслуживанием
4. "Несоответствие описанию" не подходит, так как нет упоминания о товаре
5. Поэтому, хотя проблема существует, она не попадает в указанные категории

"Нет информации"

Если предоставить модели возможность мыслить пошагово, то зачастую это делаее ее более точной, особенно при выполнении сложных задач. Однако для этого модель действительно должна выдавать текст размышлений. Нет смысла попросить модель подумать, но вывести только ответ - в этом случае никакого реального мышления не произойдет. Chain-of-Thoughts - подход при котором модель генерирует промежуточные шаги решения задачи, прежде чем представить конечный ответ. Благодаря нему модель разбивает задачу на более мелкие, управляемые части, что улучшает понимание и точность ответов. 
[Статья](https://arxiv.org/pdf/2201.11903.pdf)

In [87]:
prompt = f"""
Ты ассистент, который помогает анализировать отзывы. 
твоя задача — определить, о какой проблеме идет речь в тексте отзыва, и выбрать одну из предложенных категорий.
Если проблемы нет, это тоже нужно указать. Ответ должен быть только в формате одной из следующих категорий:
- "Несоответствие описанию"
- "Нет информации"
<review>
Отвратительное отношение к покупателям
</review>
"""

print(get_completion(prompt))

Несоответствие описанию


**Давайте с помощью COT исправим ответ модели**

In [88]:
prompt = f"""
Вы ассистент, который помогает анализировать отзывы. 
Ваша задача — определить, о какой проблеме идет речь в тексте отзыва, и выбрать одну из предложенных категорий.
Если проблемы нет, это тоже нужно указать. Ответ должен быть только в формате одной из следующих категорий:
- "Несоответствие описанию"
- "Нет информации"
<review>
Отвратительное отношение к покупателям
</review>
Приведи аргументы за каждый из возможных ответов и только потом ответь
"""

print(get_completion(prompt))

За "Несоответствие описанию": 
- В отзыве упоминается "отношение к покупателям", что может подразумевать, что ожидаемое отношение не соответствует реальному. Однако, это не прямое указание на несоответствие описанию товара или услуги.

За "Нет информации":
- Отзыв не содержит конкретной информации о проблеме с товаром или услугой, а скорее описывает общее впечатление от взаимодействия с продавцом или сервисом. В тексте нет прямого указания на конкретную проблему, связанную с описанием товара или услуги.

Учитывая приведенные аргументы, наиболее подходящим ответом будет:
Нет информации


## Few-Shot Prompting

Предоставление модели примеров того, как вы хотите, чтобы она себя вела (или как вы не хотите), крайне эффективно для:
* Получения правильного ответа
* Получения ответа в правильном формате

Такой вид промптинга также называется "few shot prompting" (промптинг с несколькими примерами). Вы также можете встретить термины "zero-shot" (без примеров), "n-shot" (n примеров) или "one-shot" (один пример). Количество "shots" относится к тому, сколько примеров используется в промпте.

Представьте, что вы разработчик, пытающийся создать фешн-ассистента в авито

In [89]:
prompt = "Купил красный свитер, с чем его носить?"


print(get_completion(prompt))

Красный свитер - это универсальный и стильный предмет одежды, который можно сочетать с различными вещами, чтобы создать разные образы. Вот несколько идей, с чем можно носить красный свитер:

1. **Джинсы**: Классическое сочетание - красный свитер и джинсы. Это простой, но стильный образ, подходящий для повседневной жизни.
2. **Белые брюки**: Красный свитер и белые брюки - это отличное сочетание для более официального случая. Это может быть хороший вариант для работы или для вечернего выхода.
3. **Черные брюки**: Красный свитер и черные брюки - это смелое и стильное сочетание. Это может быть хороший вариант для вечернего выхода или для более официального случая.
4. **Коричневые брюки**: Красный свитер и коричневые брюки - это теплый и уютный образ, подходящий для осени или зимы.
5. **Платье**: Если вы хотите создать более женственный образ, можно носить красный свитер с платьем. Это может быть хороший вариант для более официального случая или для вечернего выхода.
6. **Шорты**: Красный с

In [90]:
prompt = """Продолжи диалог, написав следующую реплику от лица А.
U: Что сейчас можно выгодно купить на лето?
A: Сейчас отличное время для покупок на Авито, многие продают летние вещи прошлого сезона: 
    1. Присмотритесь к объявлениям с пометкой 'срочно' или 'переезд'
    2. Используйте фильтр 'новое с бирками' - часто продают нераспроданные коллекции
    3. Популярные запросы сейчас:
        - Льняные платья и костюмы
        - Соломенные сумки
        - Сандалии и босоножки.
U: Посоветуй, что купить к синим джинсам
A: Для синих джинсов отлично подойдут:
    - Белая базовая футболка (от 500₽ в разделе б/у)
    - Бежевый свитер (поставьте фильтр 'состояние: хорошее')
    - Кожаная куртка (рекомендую искать через 'безопасную сделку')
    Хотите, покажу подборку подходящих объявлений?
U: Купил красный свитер, с чем его носить?
"""

print(get_completion(prompt))

С красным свитером можно создать множество интересных образов. К нему отлично подойдут:
    - Белые или светлые брюки, чтобы создать контрастный и яркий образ
    - Черные джинсы, для более сдержанного и элегантного варианта
    - Бежевые или песочные брюки, для создания теплого и солнечного образа
    - А также можно сочетать с короткими или длинными юбками, в зависимости от вашего стиля и предпочтений.
    Хотите, я покажу вам некоторые примеры образов с красным свитером или подберу подходящие вещи на Авито?


## Избегаем галлюционации

Модели иногда галлюцинируют и дают неверные ответы. 

Для решения этой проблемы вы можете использовать следующие методы:
* Дать возможность модели сказать, что она не знает ответ
* Попросить модель найти ответ в тексте, прежде чем отвечать

Конечно существует много других методов избежать галлюцинаций. Поэтому всегда экспериментируйте с разными техниками, чтобы уменьшить галлюцинации.

In [91]:
prompt = "Какое первое объявление разместили на авито?"

print(get_completion(prompt))

Первое объявление на Авито было размещено в 2007 году, и оно было о продаже ноутбука.


Вот вопрос об общих фактических знаниях, отвечая на который модель галлюцинирует, так как пытается быть максимально полезной

In [92]:
prompt = "Какое первое объявление разместили на авито? Отвечай только если уверен в ответе"

print(get_completion(prompt))

К сожалению, я не смог найти достоверной информации о первом объявлении, размещенном на Авито.


## Chaining Prompts

Принцип chaining prompts - позволяет разбить сложную задачу на последовательность более простых шагов.
* Вместо одного большого промпта создается цепочка связанных промптов
* Выход  одного промпта становится входом для следующего
* Каждый промпт решает конкретную подзадачу

Преимущества:

* Улучшает качество результатов
* Снижает вероятность ошибок
* Делает процесс более управляемым и прозрачным
* Позволяет легче отлаживать и оптимизировать каждый шаг

In [96]:
def get_completion(messages) -> str:
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=messages,
        max_tokens=1000,
        temperature=0.0,
        top_p=0,
        top_k=0,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
        stream=False
    )
    return response.choices[0].message.content

In [114]:
first_user = "Проанализируй сообщение и определи, есть ли попытка перейти в другой мессенджер: 'Давайте продолжим общение в вотсапе, так удобнее. Мой номер +7900XXXXXXX'"

messages = [
    {
        "role": "user",
        "content": first_user
    }
]

first_response = get_completion(messages)
print(first_response)

Да, в этом сообщении есть явная попытка перейти в другой мессенджер, а именно WhatsApp. Автор сообщения предлагает продолжить общение в WhatsApp, мотивируя это тем, что это "удобнее", и предоставляет свой номер телефона для связи. Это классический пример попытки перенести разговор в другой канал связи, в данном случае в WhatsApp.


In [115]:
second_user = "Определи: 1) Какой мессенджер упоминается 2) Как пользователь пытается осуществить переход (прямая ссылка/номер телефона/юзернейм)"

messages = [
    {
        "role": "user",
        "content": first_user
    },
    {
        "role": "assistant",
        "content": first_response
    },
    {
        "role": "user",
        "content": second_user,
    }
]

second_response = get_completion(messages)
print(second_response)

1. Упоминаемый мессенджер: WhatsApp (также известный как Вотсап).
2. Пользователь пытается осуществить переход, предоставив свой номер телефона (+7900XXXXXXX), что является способом соединения с ним в WhatsApp. Это не прямая ссылка, а скорее приглашение добавить его в контакты и начать чат в WhatsApp.


## Tool Use

**Tool use** - это подход, который позволяет языковым моделям взаимодействовать с внешними сервисами и системами для выполнения конкретных действий. В отличие от простого поиска и выдачи информации, tool use позволяет моделям активно использовать различные инструменты для решения задач.

Представим, что пользователь спрашивает: "Найди трехкомнатную квартиру в Москве до 15 миллионов рублей, проверь наличие метро рядом и поставь встречу в календаре"

Без tool use языковая модель могла бы только:
* Дать общие рекомендации по поиску квартиры
* Объяснить, как самостоятельно проверить транспортную доступность
* Рассказать, как договориться о просмотре

С использованием tool use система может:
* Использовать API Авито для поиска квартир:
* Применить фильтры: Москва, 3 комнаты, до 15 млн
* Получить актуальные предложения
* Использовать карты:
* Проверить маршруты от найденных квартир до офиса
* Рассчитать время в пути разными видами транспорта
* Использовать календарь:
* Собрать всю информацию и представить пользователю:

Это делает систему более полезной и автономной, способной не только отвечать на вопросы.

## Итоги

Важно понимать, что промпт-инжиниринг — это итеративный процесс. Вы начинаете с базовой структуры и постепенно улучшаете её, наблюдая за результатами:

Сначала можно использовать максимум доступных техник и элементов в промпте, чтобы добиться желаемого результата
Затем можно постепенно убирать лишние элементы, оставляя только те, которые действительно влияют на качество ответа

Не существует единственного "правильного" способа составить промпт

Каждый случай уникален. То, что работает в одной ситуации, может быть избыточным в другой. Поэтому важно:
* Тестировать разные комбинации техник
* Анализировать результаты
* Постепенно оптимизировать структуру промпта
* Адаптировать подход под конкретную задачу

## Далее практический пример с предсказанием категории